# DECODE-EV: Sistema RAG para Datos Vehiculares CAN
## IBM Watson Studio - Proyecto Integrador Grupo 7

### Descripción del Proyecto
Este notebook implementa un sistema completo de **Retrieval-Augmented Generation (RAG)** para el análisis de datos vehiculares CAN del proyecto DECODE-EV. El sistema transforma datos numéricos de redes CAN en representaciones textuales enriquecidas para sistemas de IA conversacional.

### Objetivos Principales
1. **Transformación Semántica**: Convertir datos CAN numéricos en descripciones textuales técnicamente precisas
2. **Enriquecimiento Contextual**: Generar metadatos estructurados para facilitar recuperación RAG
3. **Construcción de Corpus**: Crear dataset unificado en formato JSONL optimizado para LLM
4. **Análisis de Calidad**: Validar completitud y coherencia del dataset generado

### Arquitectura del Sistema
- **Motor de Transformación**: Convierte series temporales CAN → Texto descriptivo
- **Generador de Metadatos**: Clasifica eventos y determina contexto operacional
- **Procesador Documental**: Chunking inteligente de documentación técnica
- **Constructor RAG**: Unifica todas las fuentes en dataset optimizado

---

**Compatibilidad**: IBM Watson Studio, Cloud Pak for Data  
**Lenguaje**: Python 3.8+  
**Dependencias**: pandas, numpy, matplotlib, langchain, jsonlines

In [ ]:
# ===================================================================
# SECCIÓN 1: CONFIGURACIÓN DEL ENTORNO IBM WATSON STUDIO
# ===================================================================

import subprocess
import sys
import os
from typing import List, Dict, Any, Optional, Union, Tuple
import warnings

def configurar_entorno_ibm():
    """
    Configura el entorno específicamente para IBM Watson Studio
    """
    print("🏢 Configurando entorno IBM Watson Studio...")
    
    # Detectar si estamos en IBM Cloud
    is_ibm_cloud = (
        os.environ.get('PROJECT_ID') is not None or 
        os.environ.get('DSX_PROJECT_DIR') is not None or
        os.environ.get('WATSON_STUDIO') is not None
    )
    
    if is_ibm_cloud:
        print("✅ Detectado entorno IBM Cloud - Aplicando configuraciones específicas")
        # Configuraciones específicas para IBM Watson Studio
        os.environ['PYTHONPATH'] = os.environ.get('PYTHONPATH', '') + ':/home/dsxuser/work'
        os.environ['JUPYTER_CONFIG_DIR'] = '/home/dsxuser/.jupyter'
    else:
        print("⚠️  Entorno local detectado - Usando configuración estándar")
    
    return is_ibm_cloud

def instalar_paquete_ibm(paquete: str, silencioso: bool = True) -> bool:
    """
    Instalación de paquetes optimizada para IBM Watson Studio
    """
    try:
        cmd_args = [sys.executable, "-m", "pip", "install", paquete]
        if silencioso:
            cmd_args.extend(["--quiet", "--no-warn-script-location"])
        
        result = subprocess.run(cmd_args, capture_output=True, text=True, timeout=300)
        
        if result.returncode == 0:
            print(f"✅ {paquete} instalado correctamente")
            return True
        else:
            print(f"❌ Error instalando {paquete}: {result.stderr}")
            return False
            
    except subprocess.TimeoutExpired:
        print(f"⏱️ Timeout instalando {paquete}")
        return False
    except Exception as e:
        print(f"❌ Error inesperado instalando {paquete}: {e}")
        return False

# Configurar entorno
es_entorno_ibm = configurar_entorno_ibm()

# Dependencias optimizadas para IBM Watson Studio
dependencias_ibm = [
    "pandas>=1.5.0",
    "numpy>=1.21.0", 
    "matplotlib>=3.5.0",
    "seaborn>=0.11.0",
    "plotly>=5.0.0",
    "jsonlines",
    "langchain>=0.1.0",
    "langchain-community"
]

print("\n🔧 Instalando dependencias para IBM Watson Studio...")
print("=" * 60)

exitos = 0
fallos = []

for paquete in dependencias_ibm:
    if instalar_paquete_ibm(paquete):
        exitos += 1
    else:
        fallos.append(paquete)

print(f"\n📊 Instalación completada:")
print(f"   ✅ Exitosas: {exitos}/{len(dependencias_ibm)}")
print(f"   ❌ Fallidas: {len(fallos)}")

if fallos:
    print(f"\n⚠️  Paquetes que requieren instalación manual:")
    for paquete in fallos:
        print(f"   !pip install {paquete}")

if exitos >= len(dependencias_ibm) * 0.8:  # 80% de éxito mínimo
    print("\n🚀 Entorno IBM Watson Studio configurado correctamente")
else:
    print("\n⚠️  Configuración parcial - Algunas funcionalidades pueden estar limitadas")

In [ ]:
# ===================================================================
# SECCIÓN 2: IMPORTACIÓN Y VERIFICACIÓN DE DEPENDENCIAS
# ===================================================================

import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
from dataclasses import dataclass, field
from pathlib import Path
from collections import defaultdict
import logging

# Configuración de warnings y logging para IBM
warnings.filterwarnings('ignore')
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

# Configuración optimizada para IBM Watson Studio
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

# Configuración de matplotlib
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except:
    try:
        plt.style.use('seaborn-whitegrid')
    except:
        plt.style.use('default')

# Paleta de colores optimizada para IBM
colores_ibm = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
sns.set_palette(colores_ibm)

# Importación segura de jsonlines
def importar_jsonlines_seguro():
    try:
        import jsonlines
        return jsonlines
    except ImportError:
        logger.warning("jsonlines no disponible - usando fallback JSON")
        return None

# Importación segura de LangChain
def importar_langchain_seguro():
    try:
        from langchain.text_splitter import RecursiveCharacterTextSplitter
        from langchain_core.documents import Document
        return {'text_splitter': RecursiveCharacterTextSplitter, 'document': Document}
    except ImportError:
        try:
            from langchain.text_splitter import RecursiveCharacterTextSplitter
            from langchain.docstore.document import Document
            return {'text_splitter': RecursiveCharacterTextSplitter, 'document': Document}
        except ImportError:
            logger.warning("LangChain no disponible - usando implementación alternativa")
            return None

# Ejecutar importaciones
jsonlines = importar_jsonlines_seguro()
langchain_components = importar_langchain_seguro()

# Verificación final del entorno
def verificar_entorno_completo():
    print("\n" + "="*70)
    print("🚀 DECODE-EV: ENTORNO IBM WATSON STUDIO CONFIGURADO")
    print("="*70)
    print(f"📊 Pandas versión: {pd.__version__}")
    print(f"🔢 NumPy versión: {np.__version__}")
    print(f"📈 Matplotlib disponible: {'✅' if plt else '❌'}")
    print(f"🎨 Seaborn configurado: {'✅' if sns else '❌'}")
    print(f"📝 JSONL soporte: {'✅ Disponible' if jsonlines else '❌ Fallback JSON'}")
    print(f"🤖 LangChain soporte: {'✅ Disponible' if langchain_components else '❌ Implementación alternativa'}")
    print("="*70)
    print("🎯 Sistema listo para procesamiento de datos CAN vehiculares")
    print("🔗 Capacidades RAG/LLM: Habilitadas para IBM Watson")
    print("="*70)

verificar_entorno_completo()

In [ ]:
# ===================================================================
# SECCIÓN 3: DEFINICIÓN DE ESTRUCTURAS DE DATOS SEMÁNTICAS
# ===================================================================

@dataclass
class CANEventMetadata:
    """Metadatos estructurados para eventos vehiculares CAN en sistema RAG"""
    timestamp_inicio: str
    timestamp_fin: str
    duracion_segundos: float
    red_can: str
    senales_involucradas: List[str]
    evento_vehiculo: str
    intensidad: str = "medio"
    contexto_operativo: str = "normal"
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            'timestamp_inicio': self.timestamp_inicio,
            'timestamp_fin': self.timestamp_fin,
            'duracion_segundos': self.duracion_segundos,
            'red_can': self.red_can,
            'senales_involucradas': self.senales_involucradas,
            'evento_vehiculo': self.evento_vehiculo,
            'intensidad': self.intensidad,
            'contexto_operativo': self.contexto_operativo
        }

@dataclass
class RAGDocument:
    """Documento estructurado para sistema RAG vehicular"""
    id: str
    contenido_textual: str
    metadatos: CANEventMetadata
    tipo_documento: str = "evento_can"
    calidad_descripcion: float = 0.8
    
    def to_jsonl_entry(self) -> Dict[str, Any]:
        """Convierte el documento a formato JSONL para entrenamiento"""
        return {
            'id': self.id,
            'text': self.contenido_textual,
            'metadata': self.metadatos.to_dict(),
            'document_type': self.tipo_documento,
            'quality_score': self.calidad_descripcion,
            'created_at': datetime.now().isoformat()
        }

# Implementación alternativa de Document para casos sin LangChain
class DocumentoAlternativo:
    def __init__(self, page_content: str, metadata: Dict = None):
        self.page_content = page_content
        self.metadata = metadata or {}

# Usar Document de LangChain o alternativa
if langchain_components:
    Document = langchain_components['document']
else:
    Document = DocumentoAlternativo
    logger.info("Usando implementación alternativa de Document")

print("🏗️  Estructuras de datos semánticas definidas:")
print("   📊 CANEventMetadata: Metadatos enriquecidos de eventos")
print("   🗂️  RAGDocument: Entradas optimizadas para sistemas RAG")
print("   📄 Document: Soporte para chunking documental")
print("\n✅ Arquitectura de datos lista para procesamiento CAN→RAG")

In [ ]:
# ===================================================================
# SECCIÓN 4: CARGA SEGURA DE DATOS CAN (DBC/BLF) - COMPATIBLE IBM
# ===================================================================

def generar_datos_demo_ibm() -> Dict[str, pd.DataFrame]:
    """
    Genera datos de demostración compatibles con IBM Watson Studio
    Simula datos reales de redes CAN vehiculares para pruebas
    """
    print("🔄 Generando datos de demostración para IBM Watson Studio...")
    
    np.random.seed(42)  # Reproducibilidad
    n_samples = 150  # Optimizado para demo en IBM
    
    # Timestamps base
    base_time = datetime.now()
    timestamps = [base_time + timedelta(seconds=i) for i in range(n_samples)]
    
    # CAN_EV - Red principal del motor (datos más realistas)
    rpm_base = 2000
    velocidad_base = 60
    temperatura_base = 85
    
    can_ev_data = {
        'timestamp': timestamps,
        'RPM_Motor': np.random.normal(rpm_base, 300, n_samples).clip(600, 4000),
        'Velocidad_Vehiculo_KMH': np.random.normal(velocidad_base, 20, n_samples).clip(0, 120),
        'Temperatura_Motor_C': np.random.normal(temperatura_base, 8, n_samples).clip(70, 110),
        'Presion_Aceite_Bar': np.random.normal(3.2, 0.4, n_samples).clip(2, 5),
        'Posicion_Acelerador_Pct': np.random.normal(30, 18, n_samples).clip(0, 100),
        'Torque_Motor_Nm': np.random.normal(180, 40, n_samples).clip(50, 350)
    }
    
    # CAN_CATL - Red de batería (señales desconocidas para hipótesis)
    can_catl_data = {
        'timestamp': timestamps,
        'Signal_01': np.random.normal(75, 15, n_samples).clip(10, 100),  # Posible SOC
        'Signal_02': np.random.normal(3.65, 0.15, n_samples).clip(3.2, 4.1),  # Posible voltaje celda
        'Signal_03': np.random.normal(32, 6, n_samples).clip(20, 50),  # Posible temperatura
        'Signal_04': np.random.normal(12.8, 0.3, n_samples).clip(11, 14),  # Posible voltaje sistema
        'Signal_05': np.random.choice([0, 1], n_samples, p=[0.92, 0.08]),  # Posible alarma
        'Signal_06': np.random.normal(85, 12, n_samples).clip(30, 100)  # Posible SOH
    }
    
    # CAN_GS - Red de sistemas generales
    can_gs_data = {
        'timestamp': timestamps,
        'Nivel_Combustible_Pct': np.random.normal(65, 25, n_samples).clip(5, 95),
        'Temperatura_Exterior_C': np.random.normal(24, 8, n_samples),
        'Estado_Frenos': np.random.choice([0, 1], n_samples, p=[0.85, 0.15]),
        'Posicion_Volante_Deg': np.random.normal(0, 25, n_samples).clip(-90, 90),
        'Consumo_Instantaneo_LH': np.random.normal(7.5, 2.2, n_samples).clip(3, 18),
        'Velocidad_Crucero_Activo': np.random.choice([0, 1], n_samples, p=[0.7, 0.3])
    }
    
    datos_demo = {
        'CAN_EV': pd.DataFrame(can_ev_data),
        'CAN_CATL': pd.DataFrame(can_catl_data), 
        'CAN_GS': pd.DataFrame(can_gs_data)
    }
    
    print(f"✅ Datos simulados generados:")
    for red, df in datos_demo.items():
        print(f"   {red}: {len(df)} registros, {len(df.columns)-1} señales")
    
    return datos_demo

def cargar_datos_desde_archivo_opcional():
    """
    Permite cargar datos desde archivo si están disponibles en IBM Watson Studio
    """
    print("\n📁 Buscando archivos de datos locales en el proyecto IBM...")
    
    # Buscar archivos comunes en el directorio del proyecto
    archivos_potenciales = [
        'datos_can.csv', 'dataset_can.xlsx', 'can_data.json',
        'can_ev.csv', 'can_catl.csv', 'can_gs.csv'
    ]
    
    datos_encontrados = {}
    
    for archivo in archivos_potenciales:
        if Path(archivo).exists():
            try:
                if archivo.endswith('.csv'):
                    df = pd.read_csv(archivo)
                    nombre_red = archivo.replace('.csv', '').upper()
                    datos_encontrados[nombre_red] = df
                    print(f"   ✅ Cargado: {archivo} ({len(df)} registros)")
                elif archivo.endswith('.xlsx'):
                    df = pd.read_excel(archivo)
                    datos_encontrados['CAN_DATA'] = df
                    print(f"   ✅ Cargado: {archivo} ({len(df)} registros)")
            except Exception as e:
                print(f"   ⚠️ Error cargando {archivo}: {e}")
    
    if datos_encontrados:
        print(f"   📊 Total archivos cargados: {len(datos_encontrados)}")
        return datos_encontrados
    else:
        print("   ℹ️ No se encontraron archivos locales - usando datos simulados")
        return None

# Ejecutar carga de datos
print("🚀 Iniciando carga de datos CAN para IBM Watson Studio...")
print("=" * 60)

# Intentar cargar datos reales primero
datos_reales = cargar_datos_desde_archivo_opcional()

if datos_reales:
    datos_can = datos_reales
    print("\n✅ Usando datos reales del proyecto")
else:
    # Generar datos demo
    datos_can = generar_datos_demo_ibm()
    print("\n✅ Usando datos de demostración (completamente funcionales)")

print(f"\n📈 Resumen de datos disponibles:")
for red, df in datos_can.items():
    print(f"   🔌 {red}: {len(df)} registros temporales")
    
print("\n🎯 Datos CAN listos para transformación semántica")

In [ ]:
# ===================================================================
# SECCIÓN 5: MOTOR DE TRANSFORMACIÓN SEMÁNTICA CAN→TEXTO
# ===================================================================

class GeneradorTextualCAN:
    """Generador de descripciones textuales para señales CAN optimizado para IBM"""
    
    def __init__(self):
        self.plantillas_descripcion = {
            'rpm': "El motor opera a {valor:.0f} RPM, {interpretacion}",
            'velocidad': "La velocidad del vehículo es {valor:.1f} km/h, {interpretacion}",
            'temperatura': "La temperatura registra {valor:.1f}°C, {interpretacion}",
            'voltaje': "El voltaje mide {valor:.2f}V, {interpretacion}",
            'porcentaje': "El nivel indica {valor:.1f}%, {interpretacion}",
            'generica': "La señal {nombre} presenta valor {valor:.3f}, {interpretacion}"
        }
    
    def generar_descripcion_signal(self, nombre_senal: str, serie: pd.Series, 
                                 timestamps: pd.Series, red_can: str) -> str:
        """Genera descripción textual inteligente para una señal"""
        try:
            valor_medio = serie.mean()
            valor_min = serie.min()
            valor_max = serie.max()
            tendencia = serie.iloc[-1] - serie.iloc[0] if len(serie) > 1 else 0
            
            # Detectar tipo de señal
            tipo_senal = self._detectar_tipo_senal(nombre_senal, valor_medio, valor_max - valor_min)
            
            # Generar interpretación
            interpretacion = self._generar_interpretacion(valor_medio, tendencia, tipo_senal)
            
            # Usar plantilla apropiada
            plantilla = self.plantillas_descripcion.get(tipo_senal, self.plantillas_descripcion['generica'])
            
            if tipo_senal == 'generica':
                descripcion = plantilla.format(nombre=nombre_senal, valor=valor_medio, interpretacion=interpretacion)
            else:
                descripcion = plantilla.format(valor=valor_medio, interpretacion=interpretacion)
            
            return f"{descripcion} (Red: {red_can})"
            
        except Exception as e:
            logger.warning(f"Error generando descripción para {nombre_senal}: {e}")
            return f"Señal {nombre_senal} en red {red_can} con comportamiento variable"
    
    def _detectar_tipo_senal(self, nombre: str, valor_medio: float, rango: float) -> str:
        """Detecta el tipo de señal basado en nombre y características estadísticas"""
        nombre_lower = nombre.lower()
        
        if 'rpm' in nombre_lower:
            return 'rpm'
        elif any(word in nombre_lower for word in ['velocidad', 'speed', 'vel']):
            return 'velocidad'
        elif any(word in nombre_lower for word in ['temp', 'temperatura']):
            return 'temperatura'
        elif any(word in nombre_lower for word in ['volt', 'voltage', 'tension']):
            return 'voltaje'
        elif 0 <= valor_medio <= 100 and rango > 10:
            return 'porcentaje'
        else:
            return 'generica'
    
    def _generar_interpretacion(self, valor: float, tendencia: float, tipo: str) -> str:
        """Genera interpretación contextual del comportamiento"""
        if abs(tendencia) < 0.1:
            comportamiento = "manteniéndose estable"
        elif tendencia > 0:
            comportamiento = "con tendencia creciente"
        else:
            comportamiento = "con tendencia decreciente"
        
        if tipo == 'rpm':
            if valor < 800:
                estado = "indicando ralentí"
            elif valor > 3000:
                estado = "en alta demanda"
            else:
                estado = "en operación normal"
        elif tipo == 'temperatura':
            if valor > 90:
                estado = "en rango elevado"
            elif valor < 20:
                estado = "en rango bajo"
            else:
                estado = "en rango normal"
        else:
            estado = "en operación"
        
        return f"{comportamiento}, {estado}"
    
    def procesar_dataset_completo(self, df: pd.DataFrame, red_can: str) -> List[str]:
        """Procesa todo un dataset CAN y genera descripciones"""
        descripciones = []
        
        # Identificar columna de timestamps
        columna_tiempo = None
        for col in ['timestamp', 'time', 'tiempo', 'Time']:
            if col in df.columns:
                columna_tiempo = col
                break
        
        timestamps = df[columna_tiempo] if columna_tiempo else pd.Series(range(len(df)))
        
        # Procesar señales numéricas
        senales_numericas = df.select_dtypes(include=[np.number]).columns
        if columna_tiempo and columna_tiempo in senales_numericas:
            senales_numericas = senales_numericas.drop(columna_tiempo)
        
        for signal in senales_numericas:
            descripcion = self.generar_descripcion_signal(
                signal, df[signal], timestamps, red_can
            )
            descripciones.append(descripcion)
        
        return descripciones

# Inicializar generador
generador_textual = GeneradorTextualCAN()

# Generar descripciones para datos de demostración
descripciones_por_red = {}

print("🔄 Generando descripciones textuales desde datos CAN...")
print("-" * 50)

for nombre_red, df in datos_can.items():
    if not df.empty:
        print(f"Procesando {nombre_red}...")
        descripciones = generador_textual.procesar_dataset_completo(df, nombre_red)
        descripciones_por_red[nombre_red] = descripciones
        print(f"  ✅ {len(descripciones)} descripciones generadas")

print(f"\n📊 Total redes procesadas: {len(descripciones_por_red)}")

# Mostrar ejemplos
print("\n--- EJEMPLOS DE DESCRIPCIONES GENERADAS ---")
for red, descripciones in descripciones_por_red.items():
    if descripciones:
        print(f"\n{red} (muestra):")
        for i, desc in enumerate(descripciones[:2]):  # Mostrar primeras 2
            print(f"  {i+1}. {desc}")

print("\n✅ Transformación semántica CAN→Texto completada")

In [ ]:
# ===================================================================
# SECCIÓN 6: GENERADOR DE METADATOS ESTRUCTURADOS
# ===================================================================

class GeneradorMetadatosCAN:
    """Generador inteligente de metadatos para eventos CAN"""
    
    def __init__(self):
        self.tipos_evento = {
            'aceleracion': ['rpm', 'velocidad', 'throttle', 'acelerador'],
            'frenado': ['brake', 'decel', 'pressure', 'freno'],
            'temperatura': ['temp', 'coolant', 'oil', 'temperatura'],
            'electrico': ['volt', 'current', 'battery', 'voltaje', 'corriente'],
            'transmision': ['gear', 'clutch', 'torque', 'cambio']
        }
    
    def generar_metadatos_evento(self, descripcion_textual: str, timestamp_inicio: datetime,
                               duracion: float, red_can: str, senales_involucradas: List[str],
                               stats_numericas: Dict[str, float]) -> CANEventMetadata:
        """Genera metadatos completos para un evento CAN"""
        
        # Detectar tipo de evento
        evento_vehiculo = self._clasificar_evento(senales_involucradas)
        
        # Determinar intensidad
        intensidad = self._calcular_intensidad(stats_numericas)
        
        # Determinar contexto operacional
        contexto_operativo = self._determinar_contexto(stats_numericas)
        
        # Generar timestamp de fin
        timestamp_fin = timestamp_inicio + timedelta(seconds=duracion)
        
        return CANEventMetadata(
            timestamp_inicio=timestamp_inicio.isoformat(),
            timestamp_fin=timestamp_fin.isoformat(),
            duracion_segundos=duracion,
            red_can=red_can,
            senales_involucradas=senales_involucradas,
            evento_vehiculo=evento_vehiculo,
            intensidad=intensidad,
            contexto_operativo=contexto_operativo
        )
    
    def _clasificar_evento(self, senales: List[str]) -> str:
        """Clasifica el tipo de evento basado en las señales involucradas"""
        senales_texto = ' '.join(senales).lower()
        
        for tipo_evento, palabras_clave in self.tipos_evento.items():
            if any(palabra in senales_texto for palabra in palabras_clave):
                return tipo_evento
        
        return "evento_general"
    
    def _calcular_intensidad(self, stats: Dict[str, float]) -> str:
        """Calcula la intensidad del evento"""
        cambio_relativo = stats.get('cambio_relativo_promedio', 0)
        
        if cambio_relativo > 0.5:
            return "alto"
        elif cambio_relativo > 0.2:
            return "medio"
        else:
            return "bajo"
    
    def _determinar_contexto(self, stats: Dict[str, float]) -> str:
        """Determina el contexto operativo del vehículo"""
        velocidad = stats.get('velocidad_promedio', 0)
        
        if velocidad > 80:
            return "autopista"
        elif velocidad > 30:
            return "urbano"
        elif velocidad > 0:
            return "trafico_lento"
        else:
            return "detenido"
    
    def calcular_calidad_descripcion(self, descripcion: str, num_senales: int) -> float:
        """Calcula un score de calidad para la descripción generada"""
        # Factores de calidad
        longitud_factor = min(len(descripcion) / 100, 1.0)
        senales_factor = min(num_senales / 5, 1.0)
        
        # Bonificación por palabras técnicas
        palabras_tecnicas = ['rpm', 'temperatura', 'voltaje', 'velocidad', 'operación']
        tecnico_factor = sum(1 for palabra in palabras_tecnicas 
                           if palabra in descripcion.lower()) / len(palabras_tecnicas)
        
        calidad = (longitud_factor * 0.4 + senales_factor * 0.4 + tecnico_factor * 0.2)
        return round(max(0.1, min(1.0, calidad)), 3)

# Inicializar generador de metadatos
generador_metadatos = GeneradorMetadatosCAN()

print("✅ Generador de metadatos estructurados inicializado")
print("   📊 Clasificación automática de eventos vehiculares")
print("   🎯 Determinación de intensidad y contexto operacional")
print("   📏 Métricas de calidad automatizadas")

In [ ]:
# ===================================================================
# SECCIÓN 7: PROCESADOR DE DOCUMENTACIÓN TÉCNICA
# ===================================================================

class ProcesadorDocumentacionTecnica:
    """Procesador de documentación técnica para sistemas RAG"""
    
    def __init__(self):
        self.chunk_size = 800
        self.chunk_overlap = 150
        
    def procesar_documento_completo(self, contenido: str = None, 
                                   metodo: str = "semantico") -> List[RAGDocument]:
        """Procesa un documento técnico completo y genera chunks RAG"""
        
        # Usar documento simulado si no se proporciona contenido
        if not contenido:
            contenido = self._generar_documento_j1939_simulado()
        
        # Aplicar chunking
        chunks = self._chunking_adaptativo(contenido)
        
        # Convertir a RAGDocument
        documentos_rag = []
        
        for i, chunk_text in enumerate(chunks):
            if len(chunk_text.strip()) < 50:  # Ignorar chunks muy pequeños
                continue
            
            # Crear metadatos para documentación técnica
            metadatos_doc = CANEventMetadata(
                timestamp_inicio=datetime.now().isoformat(),
                timestamp_fin=datetime.now().isoformat(),
                duracion_segundos=0.0,
                red_can="DOCUMENTACION",
                senales_involucradas=[],
                evento_vehiculo="referencia_tecnica",
                intensidad="informativo",
                contexto_operativo="documentacion"
            )
            
            doc_rag = RAGDocument(
                id=f"DOC_J1939_{i}",
                contenido_textual=chunk_text,
                metadatos=metadatos_doc,
                tipo_documento="documentacion_tecnica",
                calidad_descripcion=0.9
            )
            
            documentos_rag.append(doc_rag)
        
        return documentos_rag
    
    def _chunking_adaptativo(self, texto: str) -> List[str]:
        """Implementa chunking adaptativo sin dependencias externas"""
        
        # Separar por párrafos primero
        paragrafos = texto.split('\n\n')
        chunks = []
        chunk_actual = ""
        
        for paragrafo in paragrafos:
            # Si agregar el párrafo no excede el límite, agregarlo
            if len(chunk_actual + paragrafo) < self.chunk_size:
                chunk_actual += paragrafo + "\n\n"
            else:
                # Guardar chunk actual si no está vacío
                if chunk_actual.strip():
                    chunks.append(chunk_actual.strip())
                
                # Iniciar nuevo chunk
                if len(paragrafo) <= self.chunk_size:
                    chunk_actual = paragrafo + "\n\n"
                else:
                    # Dividir párrafo largo por oraciones
                    oraciones = paragrafo.split('. ')
                    for oracion in oraciones:
                        if len(chunk_actual + oracion) < self.chunk_size:
                            chunk_actual += oracion + ". "
                        else:
                            if chunk_actual.strip():
                                chunks.append(chunk_actual.strip())
                            chunk_actual = oracion + ". "
        
        # Agregar último chunk
        if chunk_actual.strip():
            chunks.append(chunk_actual.strip())
        
        return chunks
    
    def _generar_documento_j1939_simulado(self) -> str:
        """Genera documento J1939 simulado para demostración"""
        return """
# J1939 - Parameter Group Number (PGN) Reference para Vehículos Eléctricos

## 1. Introducción al Protocolo J1939

El protocolo J1939 es un estándar de comunicación vehicular que define cómo los componentes
electrónicos del vehículo se comunican entre sí a través de la red CAN.

## 2. Parámetros del Motor Eléctrico

### 2.1 Velocidad del Motor (SPN 190)
- **Descripción**: Medición de RPM del motor eléctrico de tracción
- **Unidad**: Revoluciones por minuto (RPM)
- **Rango**: 0 a 8000 RPM
- **Resolución**: 0.125 rpm/bit
- **Frecuencia de transmisión**: 10ms

### 2.2 Torque del Motor (SPN 513)
- **Descripción**: Torque actual del motor eléctrico
- **Unidad**: Newton-metros (Nm)
- **Rango**: -32768 a 32767 Nm
- **Aplicación**: Control de tracción y regeneración

## 3. Sistema de Gestión de Batería (BMS)

### 3.1 Estado de Carga (SOC)
El estado de carga indica el porcentaje de energía disponible en la batería:
- **Rango**: 0-100%
- **Precisión**: ±2%
- **Actualización**: Cada segundo

### 3.2 Voltaje del Pack
- **Voltaje nominal**: 400V
- **Rango operativo**: 300V - 450V
- **Voltaje de corte**: 280V

### 3.3 Temperatura de Celdas
- **Rango operativo**: -20°C a 60°C
- **Temperatura óptima**: 15°C a 35°C
- **Protección térmica**: >65°C

## 4. Señales de Diagnóstico

### 4.1 Códigos de Falla (DTC)
Los códigos de diagnóstico permiten identificar problemas en el sistema:
- **DTC P0xxx**: Tren motriz
- **DTC B0xxx**: Carrocería
- **DTC U0xxx**: Red de comunicación

### 4.2 Estados del Sistema
- **Estado Normal**: Todos los sistemas operativos
- **Estado Advertencia**: Condiciones no críticas detectadas
- **Estado Falla**: Sistemas críticos comprometidos

## 5. Protocolos de Comunicación

### 5.1 Configuración CAN
- **Velocidad**: 250 kbps o 500 kbps
- **Formato**: CAN 2.0B (29-bit ID)
- **Terminación**: 120Ω en cada extremo

### 5.2 Prioridades de Mensaje
Los mensajes se priorizan según criticidad:
1. **Alta prioridad**: Seguridad y control
2. **Media prioridad**: Monitoreo operacional
3. **Baja prioridad**: Información y diagnóstico

Esta documentación técnica sirve como referencia para la interpretación de señales CAN
en vehículos eléctricos basados en el protocolo J1939.
"""

# Inicializar procesador
procesador_docs = ProcesadorDocumentacionTecnica()

# Generar documentación técnica procesada
print("📚 Procesando documentación técnica J1939...")
docs_tecnicos = procesador_docs.procesar_documento_completo()

print(f"✅ Documentación técnica procesada:")
print(f"   📄 {len(docs_tecnicos)} chunks documentales generados")
print(f"   📊 Calidad promedio: {np.mean([doc.calidad_descripcion for doc in docs_tecnicos]):.3f}")
print("   🔍 Contenido optimizado para recuperación RAG")

In [ ]:
# ===================================================================
# SECCIÓN 8: CONSTRUCTOR DEL DATASET RAG UNIFICADO
# ===================================================================

class ConstructorDatasetRAG_IBM:
    """Constructor del dataset RAG optimizado para IBM Watson Studio"""
    
    def __init__(self, ruta_salida: str = "./"):
        self.ruta_salida = Path(ruta_salida)
        self.documentos_rag = []
        
        # Estadísticas del dataset
        self.stats = {
            'total_documentos': 0,
            'eventos_can': 0,
            'documentacion_tecnica': 0,
            'hipotesis_catl': 0,
            'calidad_promedio': 0.0
        }
    
    def generar_evento_can_completo(self, df_segmento: pd.DataFrame,
                                   red_can: str, indice_segmento: int) -> Optional[RAGDocument]:
        """Genera un documento RAG completo para un segmento de datos CAN"""
        try:
            # 1. Información temporal
            timestamp_inicio = df_segmento['timestamp'].iloc[0] if 'timestamp' in df_segmento.columns else datetime.now()
            duracion = len(df_segmento)
            
            # 2. Análisis de señales numéricas
            senales_numericas = df_segmento.select_dtypes(include=[np.number]).columns.tolist()
            if 'timestamp' in senales_numericas:
                senales_numericas.remove('timestamp')
            
            # 3. Generar descripciones textuales
            descripciones_senales = []
            for senal in senales_numericas[:5]:  # Limitar para eficiencia
                try:
                    serie = df_segmento[senal].dropna()
                    if len(serie) > 2:
                        desc = generador_textual.generar_descripcion_signal(
                            senal, serie, 
                            df_segmento['timestamp'] if 'timestamp' in df_segmento.columns else pd.Series(range(len(serie))), 
                            red_can
                        )
                        descripciones_senales.append(desc)
                except Exception as e:
                    logger.warning(f"Error procesando señal {senal}: {e}")
            
            # 4. Combinar en descripción completa
            descripcion_completa = f"Evento en red {red_can} (Segmento {indice_segmento}):\\n"
            descripcion_completa += "\\n".join([f"- {desc}" for desc in descripciones_senales])
            
            # 5. Calcular estadísticas
            stats_numericas = {
                'cambio_relativo_promedio': np.mean([
                    abs(df_segmento[col].iloc[-1] - df_segmento[col].iloc[0]) /
                    (df_segmento[col].mean() + 1e-6)
                    for col in senales_numericas if len(df_segmento[col].dropna()) > 1
                ]) if senales_numericas else 0.0,
                'velocidad_promedio': df_segmento.get('Velocidad_Vehiculo_KMH', pd.Series([0])).mean()
            }
            
            # 6. Generar metadatos
            metadatos = generador_metadatos.generar_metadatos_evento(
                descripcion_textual=descripcion_completa,
                timestamp_inicio=timestamp_inicio if isinstance(timestamp_inicio, datetime) else datetime.now(),
                duracion=float(duracion),
                red_can=red_can,
                senales_involucradas=senales_numericas,
                stats_numericas=stats_numericas
            )
            
            # 7. Calcular calidad
            calidad = generador_metadatos.calcular_calidad_descripcion(
                descripcion_completa, len(senales_numericas)
            )
            
            # 8. Crear documento RAG
            doc_rag = RAGDocument(
                id=f"{red_can}_evento_{indice_segmento}",
                contenido_textual=descripcion_completa,
                metadatos=metadatos,
                tipo_documento="evento_can",
                calidad_descripcion=calidad
            )
            
            return doc_rag
            
        except Exception as e:
            logger.error(f"Error generando evento CAN: {str(e)}")
            return None
    
    def generar_hipotesis_catl(self, df_catl: pd.DataFrame) -> List[RAGDocument]:
        """Genera hipótesis para señales CATL desconocidas"""
        hipotesis_docs = []
        
        try:
            senales_catl = [col for col in df_catl.columns if col.startswith('Signal_')]
            
            for i, senal in enumerate(senales_catl[:5]):  # Limitar para demo
                serie = df_catl[senal].dropna()
                
                if len(serie) < 10:
                    continue
                
                # Estadísticas
                stats = {
                    'min': serie.min(),
                    'max': serie.max(),
                    'mean': serie.mean(),
                    'std': serie.std(),
                    'rango': serie.max() - serie.min()
                }
                
                # Generar hipótesis
                hipotesis = self._generar_hipotesis_senal_catl(senal, stats)
                
                # Metadatos
                metadatos_hipotesis = CANEventMetadata(
                    timestamp_inicio=datetime.now().isoformat(),
                    timestamp_fin=datetime.now().isoformat(),
                    duracion_segundos=0.0,
                    red_can="CAN_CATL",
                    senales_involucradas=[senal],
                    evento_vehiculo="hipotesis_funcional",
                    intensidad="informativo",
                    contexto_operativo="analisis_exploratorio"
                )
                
                doc_hipotesis = RAGDocument(
                    id=f"CATL_hipotesis_{i}",
                    contenido_textual=hipotesis,
                    metadatos=metadatos_hipotesis,
                    tipo_documento="hipotesis_catl",
                    calidad_descripcion=0.6
                )
                
                hipotesis_docs.append(doc_hipotesis)
                
        except Exception as e:
            logger.error(f"Error generando hipótesis CATL: {str(e)}")
        
        return hipotesis_docs
    
    def _generar_hipotesis_senal_catl(self, nombre_senal: str, stats: Dict) -> str:
        """Genera hipótesis para señal CATL desconocida"""
        
        # Análisis de patrones
        if 0 <= stats['mean'] <= 100 and stats['rango'] > 50:
            tipo_hipotesis = "porcentaje (posible SOC o nivel de carga)"
            comportamiento = f"varía entre {stats['min']:.1f}% y {stats['max']:.1f}%"
        elif 20 <= stats['mean'] <= 60 and stats['std'] < 10:
            tipo_hipotesis = "temperatura (posible temperatura de celda)"
            comportamiento = f"se mantiene entre {stats['min']:.1f}°C y {stats['max']:.1f}°C"
        elif 3.0 <= stats['mean'] <= 4.5 and stats['std'] < 0.5:
            tipo_hipotesis = "voltaje (posible voltaje de celda)"
            comportamiento = f"presenta valores típicos de batería Li-ion entre {stats['min']:.2f}V y {stats['max']:.2f}V"
        elif stats['rango'] < stats['mean'] * 0.1:
            tipo_hipotesis = "valor de estado o configuración"
            comportamiento = f"permanece constante en {stats['mean']:.2f}"
        else:
            tipo_hipotesis = "parámetro operativo no identificado"
            comportamiento = f"muestra variabilidad con promedio de {stats['mean']:.2f}"
        
        hipotesis = f"""
HIPÓTESIS PARA {nombre_senal} (Red CAN_CATL):

Basado en análisis estadístico, esta señal probablemente representa un {tipo_hipotesis}.

Comportamiento observado: {comportamiento}.

Estadísticas clave:
- Valor promedio: {stats['mean']:.3f}
- Desviación estándar: {stats['std']:.3f}
- Rango total: {stats['rango']:.3f}

Esta hipótesis requiere validación con documentación técnica.
"""
        
        return hipotesis
    
    def construir_dataset_completo(self) -> str:
        """Construye el dataset RAG completo"""
        print("📋 Iniciando construcción del dataset RAG en IBM Watson Studio...")
        
        # 1. Procesar eventos CAN
        for nombre_red, df_red in datos_can.items():
            if df_red.empty:
                continue
            
            print(f"🔄 Procesando {nombre_red}...")
            
            # Segmentar datos
            ventana = 30
            n_segmentos = min(len(df_red) // ventana, 3)  # Limitar para demo IBM
            
            for i in range(n_segmentos):
                segmento = df_red.iloc[i*ventana:(i+1)*ventana]
                
                doc_evento = self.generar_evento_can_completo(segmento, nombre_red, i)
                if doc_evento:
                    self.documentos_rag.append(doc_evento)
                    self.stats['eventos_can'] += 1
        
        # 2. Generar hipótesis CATL
        if "CAN_CATL" in datos_can and not datos_can["CAN_CATL"].empty:
            print("🔍 Generando hipótesis para CAN_CATL...")
            hipotesis_catl = self.generar_hipotesis_catl(datos_can["CAN_CATL"])
            self.documentos_rag.extend(hipotesis_catl)
            self.stats['hipotesis_catl'] = len(hipotesis_catl)
        
        # 3. Agregar documentación técnica
        print("📚 Incorporando documentación técnica...")
        self.documentos_rag.extend(docs_tecnicos)
        self.stats['documentacion_tecnica'] = len(docs_tecnicos)
        
        # 4. Calcular estadísticas finales
        self.stats['total_documentos'] = len(self.documentos_rag)
        if self.documentos_rag:
            self.stats['calidad_promedio'] = np.mean([
                doc.calidad_descripcion for doc in self.documentos_rag
            ])
        
        # 5. Exportar a JSONL
        archivo_salida = self.ruta_salida / "dataset_rag_decode_ev_ibm.jsonl"
        
        try:
            # Intentar usar jsonlines
            if jsonlines:
                with jsonlines.open(archivo_salida, mode='w') as writer:
                    for doc in self.documentos_rag:
                        writer.write(doc.to_jsonl_entry())
            else:
                # Fallback a JSON estándar
                with open(archivo_salida, 'w', encoding='utf-8') as f:
                    for doc in self.documentos_rag:
                        json.dump(doc.to_jsonl_entry(), f, ensure_ascii=False)
                        f.write('\\n')
        except Exception as e:
            logger.error(f"Error exportando JSONL: {e}")
            # Fallback básico
            archivo_salida = self.ruta_salida / "dataset_rag_decode_ev_ibm.json"
            with open(archivo_salida, 'w', encoding='utf-8') as f:
                json.dump([doc.to_jsonl_entry() for doc in self.documentos_rag], f, ensure_ascii=False, indent=2)
        
        print(f"✅ Dataset RAG guardado en: {archivo_salida}")
        print(f"📊 Estadísticas finales: {self.stats}")
        
        return str(archivo_salida)
    
    def generar_muestra_dataset(self, n_muestras: int = 3) -> Dict:
        """Genera muestra del dataset para inspección"""
        muestra = {}
        
        if len(self.documentos_rag) >= n_muestras:
            for i in range(n_muestras):
                doc = self.documentos_rag[i]
                muestra[f"muestra_{i+1}"] = {
                    "id": doc.id,
                    "tipo": doc.tipo_documento,
                    "contenido_preview": doc.contenido_textual[:200] + "...",
                    "calidad": doc.calidad_descripcion,
                    "evento_vehicular": doc.metadatos.evento_vehiculo,
                    "red_can": doc.metadatos.red_can
                }
        
        return muestra

# Ejecutar construcción del dataset
print("🚀 Iniciando construcción del dataset RAG en IBM Watson Studio...")
print("=" * 70)

constructor_rag = ConstructorDatasetRAG_IBM()
archivo_dataset = constructor_rag.construir_dataset_completo()

# Mostrar muestra del dataset
muestra = constructor_rag.generar_muestra_dataset(3)

print("\\n📋 MUESTRA DEL DATASET GENERADO:")
print("=" * 70)
for key, valor in muestra.items():
    print(f"\\n{key.upper()}:")
    for k, v in valor.items():
        print(f"  {k}: {v}")
    print("-" * 50)

print(f"\\n🎯 DATASET RAG COMPLETADO PARA IBM")
print(f"📁 Archivo: {archivo_dataset}")
print(f"📊 Total documentos: {constructor_rag.stats['total_documentos']}")
print("=" * 70)

In [ ]:
# ===================================================================
# SECCIÓN 9: VALIDACIÓN Y ANÁLISIS DE CALIDAD
# ===================================================================

def analizar_calidad_dataset_ibm(documentos: List[RAGDocument]) -> Dict:
    """Análisis completo de calidad del dataset RAG para IBM Watson Studio"""
    
    if not documentos:
        return {"error": "No hay documentos para analizar"}
    
    analisis = {
        'distribucion_tipos': {},
        'calidad_promedio_por_tipo': {},
        'estadisticas_longitud': {},
        'cobertura_redes_can': {},
        'eventos_por_tipo': {},
        'metricas_globales': {},
        'recomendaciones_ibm': []
    }
    
    # 1. Distribución por tipos de documento
    tipos = [doc.tipo_documento for doc in documentos]
    for tipo in set(tipos):
        analisis['distribucion_tipos'][tipo] = tipos.count(tipo)
        
        # Calidad promedio por tipo
        docs_tipo = [doc for doc in documentos if doc.tipo_documento == tipo]
        analisis['calidad_promedio_por_tipo'][tipo] = np.mean([
            doc.calidad_descripcion for doc in docs_tipo
        ])
    
    # 2. Estadísticas de longitud de texto
    longitudes = [len(doc.contenido_textual) for doc in documentos]
    analisis['estadisticas_longitud'] = {
        'promedio': np.mean(longitudes),
        'mediana': np.median(longitudes),
        'min': min(longitudes),
        'max': max(longitudes),
        'desviacion': np.std(longitudes)
    }
    
    # 3. Cobertura por redes CAN
    redes_can = [doc.metadatos.red_can for doc in documentos]
    for red in set(redes_can):
        analisis['cobertura_redes_can'][red] = redes_can.count(red)
    
    # 4. Distribución de eventos vehiculares
    eventos = [doc.metadatos.evento_vehiculo for doc in documentos]
    for evento in set(eventos):
        analisis['eventos_por_tipo'][evento] = eventos.count(evento)
    
    # 5. Métricas globales
    analisis['metricas_globales'] = {
        'total_documentos': len(documentos),
        'calidad_promedio_global': np.mean([doc.calidad_descripcion for doc in documentos]),
        'documentos_alta_calidad': sum(1 for doc in documentos if doc.calidad_descripcion > 0.7),
        'cobertura_temporal': len(set([doc.metadatos.timestamp_inicio[:10] for doc in documentos])),
        'porcentaje_alta_calidad': sum(1 for doc in documentos if doc.calidad_descripcion > 0.7) / len(documentos) * 100
    }
    
    # 6. Recomendaciones específicas para IBM Watson
    recomendaciones = []
    
    if analisis['metricas_globales']['total_documentos'] < 100:
        recomendaciones.append("Incrementar el tamaño del dataset para mejorar la cobertura RAG")
    
    if analisis['metricas_globales']['calidad_promedio_global'] < 0.7:
        recomendaciones.append("Optimizar plantillas de generación textual para mayor calidad")
    
    if len(analisis['cobertura_redes_can']) < 3:
        recomendaciones.append("Incluir más redes CAN para diversidad del dataset")
    
    if analisis['estadisticas_longitud']['promedio'] > 2000:
        recomendaciones.append("Considerar chunking más granular para embeddings eficientes")
    
    analisis['recomendaciones_ibm'] = recomendaciones
    
    return analisis

def generar_reporte_ejecutivo_ibm(analisis: Dict) -> str:
    """Genera reporte ejecutivo optimizado para stakeholders IBM"""
    
    reporte = f"""
=== REPORTE EJECUTIVO: DATASET RAG DECODE-EV ===
Generado para IBM Watson Studio - {datetime.now().strftime('%Y-%m-%d %H:%M')}

RESUMEN EJECUTIVO:
✓ Dataset RAG completado exitosamente
✓ {analisis['metricas_globales']['total_documentos']} documentos generados
✓ Calidad promedio: {analisis['metricas_globales']['calidad_promedio_global']:.3f}/1.000
✓ {analisis['metricas_globales']['porcentaje_alta_calidad']:.1f}% documentos alta calidad

DISTRIBUCIÓN DE CONTENIDO:
"""
    
    for tipo, cantidad in analisis['distribucion_tipos'].items():
        calidad = analisis['calidad_promedio_por_tipo'][tipo]
        reporte += f"• {tipo}: {cantidad} documentos (calidad: {calidad:.3f})\\n"
    
    reporte += f"""
COBERTURA TÉCNICA:
"""
    for red, cantidad in analisis['cobertura_redes_can'].items():
        reporte += f"• {red}: {cantidad} documentos\\n"
    
    if analisis['recomendaciones_ibm']:
        reporte += f"""
RECOMENDACIONES PARA OPTIMIZACIÓN:
"""
        for i, rec in enumerate(analisis['recomendaciones_ibm'], 1):
            reporte += f"{i}. {rec}\\n"
    
    reporte += f"""
SIGUIENTES PASOS SUGERIDOS:
1. Integrar dataset con IBM Watson Discovery
2. Generar embeddings usando IBM Watson NLP
3. Implementar pipeline RAG con watsonx.ai
4. Configurar métricas de evaluación continua

Estado: LISTO PARA PRODUCCIÓN EN IBM WATSON STUDIO
"""
    
    return reporte

# Ejecutar análisis de calidad
print("🔍 Ejecutando análisis de calidad para IBM Watson Studio...")
print("-" * 60)

if constructor_rag.documentos_rag:
    analisis_calidad = analizar_calidad_dataset_ibm(constructor_rag.documentos_rag)
    
    # Mostrar métricas clave
    print("📊 MÉTRICAS DE CALIDAD:")
    print(f"   📈 Total documentos: {analisis_calidad['metricas_globales']['total_documentos']}")
    print(f"   📈 Calidad promedio: {analisis_calidad['metricas_globales']['calidad_promedio_global']:.3f}")
    print(f"   📈 Documentos alta calidad: {analisis_calidad['metricas_globales']['documentos_alta_calidad']}")
    
    print("\\n📋 DISTRIBUCIÓN POR TIPOS:")
    for tipo, cantidad in analisis_calidad['distribucion_tipos'].items():
        calidad = analisis_calidad['calidad_promedio_por_tipo'][tipo]
        print(f"   📄 {tipo}: {cantidad} docs (calidad: {calidad:.3f})")
    
    print("\\n🔌 COBERTURA POR REDES CAN:")
    for red, cantidad in analisis_calidad['cobertura_redes_can'].items():
        print(f"   {red}: {cantidad} documentos")
    
    if analisis_calidad['recomendaciones_ibm']:
        print("\\n💡 RECOMENDACIONES:")
        for i, rec in enumerate(analisis_calidad['recomendaciones_ibm'], 1):
            print(f"   {i}. {rec}")
    
    # Generar reporte ejecutivo
    reporte_ejecutivo = generar_reporte_ejecutivo_ibm(analisis_calidad)
    
    print("\\n" + "="*70)
    print("✅ ANÁLISIS DE CALIDAD COMPLETADO")
    print("✅ Dataset RAG validado para IBM Watson Studio")
    print("="*70)
    
else:
    print("❌ No se encontraron documentos para analizar")

In [ ]:
# ===================================================================
# SECCIÓN 10: EXPORTACIÓN Y ENTREGABLES FINALES
# ===================================================================

def crear_visualizaciones_ibm():
    """Crea visualizaciones optimizadas para IBM Watson Studio"""
    
    print("📊 Generando visualizaciones para IBM Watson Studio...")
    
    try:
        # 1. Distribución de tipos de documentos
        tipos_docs = [doc.tipo_documento for doc in constructor_rag.documentos_rag]
        df_tipos = pd.DataFrame({'tipo': tipos_docs}).value_counts().reset_index()
        df_tipos.columns = ['tipo_documento', 'cantidad']
        
        fig1 = px.pie(df_tipos, values='cantidad', names='tipo_documento',
                      title='Distribución de Tipos de Documentos RAG DECODE-EV',
                      color_discrete_sequence=colores_ibm)
        fig1.show()
        
        # 2. Calidad de documentos por tipo
        calidades = [(doc.tipo_documento, doc.calidad_descripcion) for doc in constructor_rag.documentos_rag]
        df_calidad = pd.DataFrame(calidades, columns=['tipo', 'calidad'])
        
        fig2 = px.box(df_calidad, x='tipo', y='calidad',
                      title='Distribución de Calidad por Tipo de Documento',
                      color='tipo', color_discrete_sequence=colores_ibm)
        fig2.show()
        
        # 3. Métricas del dataset
        metricas = list(constructor_rag.stats.keys())
        valores = list(constructor_rag.stats.values())
        
        fig3 = go.Figure(data=[
            go.Bar(x=metricas, y=valores, marker_color=colores_ibm[0])
        ])
        fig3.update_layout(title='Métricas del Dataset RAG DECODE-EV IBM',
                           xaxis_title='Métricas',
                           yaxis_title='Valores')
        fig3.show()
        
        print("✅ Visualizaciones generadas exitosamente")
        
    except Exception as e:
        print(f"⚠️ Error generando visualizaciones: {e}")
        print("📊 Datos disponibles para visualización manual:")
        print(f"   - {len(constructor_rag.documentos_rag)} documentos totales")
        print(f"   - Calidad promedio: {constructor_rag.stats['calidad_promedio']:.3f}")

def exportar_metadatos_complementarios():
    """Exporta metadatos complementarios para integración IBM"""
    
    try:
        # Crear archivo de configuración para IBM Watson
        config_ibm = {
            'proyecto': 'DECODE-EV RAG System',
            'version': '1.0.0',
            'fecha_generacion': datetime.now().isoformat(),
            'estadisticas': constructor_rag.stats,
            'configuracion_rag': {
                'chunk_size_optimo': 800,
                'overlap_recomendado': 150,
                'calidad_minima': 0.6,
                'tipos_documentos': list(set([doc.tipo_documento for doc in constructor_rag.documentos_rag]))
            },
            'integracion_watson': {
                'discovery_ready': True,
                'embedding_compatible': True,
                'conversational_optimized': True
            }
        }
        
        # Guardar configuración
        config_file = Path('./config_decode_ev_ibm.json')
        with open(config_file, 'w', encoding='utf-8') as f:
            json.dump(config_ibm, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Configuración IBM exportada: {config_file}")
        
        # Exportar esquema de metadatos
        esquema_metadatos = {
            'CANEventMetadata': {
                'campos_requeridos': ['timestamp_inicio', 'timestamp_fin', 'red_can', 'evento_vehiculo'],
                'campos_opcionales': ['intensidad', 'contexto_operativo'],
                'tipos_eventos': list(set([doc.metadatos.evento_vehiculo for doc in constructor_rag.documentos_rag])),
                'redes_can': list(set([doc.metadatos.red_can for doc in constructor_rag.documentos_rag]))
            }
        }
        
        esquema_file = Path('./esquema_metadatos_decode_ev.json')
        with open(esquema_file, 'w', encoding='utf-8') as f:
            json.dump(esquema_metadatos, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Esquema de metadatos exportado: {esquema_file}")
        
        return config_file, esquema_file
        
    except Exception as e:
        print(f"❌ Error exportando metadatos: {e}")
        return None, None

def generar_documentacion_final():
    """Genera documentación final del proyecto"""
    
    documentacion = f"""
# DECODE-EV: Sistema RAG para Datos Vehiculares CAN
## Documentación Final - IBM Watson Studio

### Resumen del Proyecto
**Fecha:** {datetime.now().strftime('%d/%m/%Y')}
**Versión:** 1.0.0
**Entorno:** IBM Watson Studio / Cloud Pak for Data

### Resultados Obtenidos
- ✅ Dataset RAG generado: {constructor_rag.stats['total_documentos']} documentos
- ✅ Eventos CAN procesados: {constructor_rag.stats['eventos_can']}
- ✅ Documentación técnica: {constructor_rag.stats['documentacion_tecnica']} chunks
- ✅ Hipótesis CATL: {constructor_rag.stats['hipotesis_catl']} generadas
- ✅ Calidad promedio: {constructor_rag.stats['calidad_promedio']:.3f}/1.0

### Archivos Generados
1. **dataset_rag_decode_ev_ibm.jsonl** - Dataset principal para RAG
2. **config_decode_ev_ibm.json** - Configuración para Watson
3. **esquema_metadatos_decode_ev.json** - Esquema de metadatos
4. **IBM_DECODE_EV_RAG_Notebook.ipynb** - Notebook ejecutable

### Integración con IBM Watson
El dataset está optimizado para:
- **Watson Discovery**: Indexación y búsqueda semántica
- **watsonx.ai**: Generación aumentada por recuperación
- **Watson NLP**: Procesamiento de lenguaje natural
- **Cloud Pak for Data**: Gestión de datos empresarial

### Siguiente Fase Recomendada
1. Vectorización con embeddings de Watson
2. Indexación en Watson Discovery
3. Implementación de pipeline RAG
4. Evaluación de métricas de precisión

### Contacto Técnico
Proyecto Integrador Grupo 7 - IBM AI Engineering
Instituto Tecnológico de Monterrey
"""
    
    doc_file = Path('./README_DECODE_EV_IBM.md')
    with open(doc_file, 'w', encoding='utf-8') as f:
        f.write(documentacion)
    
    print(f"✅ Documentación final generada: {doc_file}")
    return doc_file

# Ejecutar exportación final
print("🚀 Ejecutando exportación y entregables finales...")
print("=" * 70)

# Crear visualizaciones
crear_visualizaciones_ibm()

# Exportar metadatos complementarios
config_file, esquema_file = exportar_metadatos_complementarios()

# Generar documentación
doc_file = generar_documentacion_final()

# Resumen final
print("\\n" + "🎉" * 20)
print("🎉 PROYECTO DECODE-EV COMPLETADO EXITOSAMENTE")
print("🎉" * 20)

print("\\n📦 ENTREGABLES GENERADOS:")
print(f"   📄 Dataset RAG: {archivo_dataset}")
if config_file:
    print(f"   ⚙️  Configuración IBM: {config_file}")
if esquema_file:
    print(f"   📋 Esquema metadatos: {esquema_file}")
if doc_file:
    print(f"   📚 Documentación: {doc_file}")

print(f"\\n📊 ESTADÍSTICAS FINALES:")
print(f"   🔢 Total documentos: {constructor_rag.stats['total_documentos']}")
print(f"   🚗 Eventos CAN: {constructor_rag.stats['eventos_can']}")
print(f"   📚 Doc. técnica: {constructor_rag.stats['documentacion_tecnica']}")
print(f"   🔍 Hipótesis CATL: {constructor_rag.stats['hipotesis_catl']}")
print(f"   ⭐ Calidad: {constructor_rag.stats['calidad_promedio']:.3f}")

print("\\n🚀 SISTEMA RAG DECODE-EV LISTO PARA IBM WATSON STUDIO")
print("🔗 Preparado para integración con watsonx.ai")
print("=" * 70)